In [ ]:
!pip install opencv-python-headless
!pip install av
!pip install nlpaug
!pip install tsaug
!pip install augly
!pip install augly[audio]
!pip install tensorflow-gpu librosa numpy matplotlib augly
!pip install -U augly[video]
!sudo apt-get install python3-magic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 15.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 13.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=cced5596cefbdfba36e31b46aa169d7b57fb0a4e6946272a11688382

In [ ]:
# Imports

import os
import random
import cv2
import numpy as np
import pandas as pd
import urllib.request
import zipfile
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Embedding, GlobalMaxPooling1D, Dropout, LSTM
from keras.preprocessing.text import Tokenizer
import augly.video as vidaugs
import nlpaug.augmenter.word as naw
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from tsaug import TimeWarp, Crop, Drift, AddNoise
from tensorflow.keras.utils import to_categorical
import tsaug
from tsaug import TimeWarp, Drift, Reverse
import nlpaug.augmenter.word as naw
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import augly.audio as audaugs
import librosa
import torch

### Images

In [ ]:
# Download dataset
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
filename = 'cats_and_dogs_filtered.zip'
urllib.request.urlretrieve(url, filename)

# Extract dataset
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('.')

In [ ]:
# Set up directories
base_dir = 'cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'validation')

# Set up data generators with data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

# Define and compile model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

# Train model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=50,
    epochs=5,
    validation_data=test_generator,
    validation_steps=50)

# Evaluate model
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('Test accuracy:', test_acc)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


<ipython-input-4-fd12485f196c>:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/5
50/50 [==============================] - 34s 370ms/step - loss: 0.7188 - accuracy: 0.5010 - val_loss: 0.6921 - val_accuracy: 0.6180
Epoch 2/5
50/50 [==============================] - 12s 247ms/step - loss: 0.6958 - accuracy: 0.5100 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 3/5
50/50 [==============================] - 11s 221ms/step - loss: 0.6919 - accuracy: 0.5200 - val_loss: 0.6905 - val_accuracy: 0.5190
Epoch 4/5
50/50 [==============================] - 11s 220ms/step - loss: 0.6924 - accuracy: 0.5430 - val_loss: 0.6646 - val_accuracy: 0.6480
Epoch 5/5
50/50 [==============================] - 11s 221ms/step - loss: 0.6807 - accuracy: 0.5910 - val_loss: 0.6538 - val_accuracy: 0.5930


<ipython-input-4-fd12485f196c>:59: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)


Test accuracy: 0.5929999947547913


### Video

In [ ]:
!wget -c http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
!unrar x hmdb51_org.rar
!unrar x shoot_gun.rar
!unrar x brush_hair.rar

--2023-04-17 08:49:32--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2023-04-17 08:49:32--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘hmdb51_org.rar’

hmdb51_org.rar      100%[===================>]   1.98G  13.5MB/s    in 2m 32s  

2023-04-17 08:52:06 (13.3 MB/s) - ‘hmdb51_org.rar’ saved [2124008126/2124008126]


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from hmdb51_org.rar


In [ ]:
data_dir_1 = './brush_hair'
data_dir_2 = './shoot_gun'
img_size = 64

# Read frames from videos in the first directory
data_1 = []
for i, filename in enumerate(os.listdir(data_dir_1)):
    if i == 5:
        break
    video_path = os.path.join(data_dir_1, filename)
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        data_1.append(frame)
    cap.release()

# Read frames from videos in the second directory
data_2 = []
for i, filename in enumerate(os.listdir(data_dir_2)):
    if i == 5:
        break
    video_path = os.path.join(data_dir_2, filename)
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        data_2.append(frame)
    cap.release()

# Create labels for the data
labels_1 = np.zeros(len(data_1))
labels_2 = np.ones(len(data_2))

# Combine the data and labels
data = np.array(data_1 + data_2)
labels = np.concatenate((labels_1, labels_2))

# Shuffle the data and labels
idx = np.random.permutation(len(data))
data = data[idx]
labels = labels[idx]

# Split the data into training and testing sets
split = int(0.8 * len(data))
x_train = data[:split]
y_train = labels[:split]
x_test = data[split:]
y_test = labels[split:]

In [ ]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
42/42 [==============================] - 3s 17ms/step - loss: 2.2786 - accuracy: 0.8908 - val_loss: 0.2113 - val_accuracy: 0.9369
Epoch 2/10
42/42 [==============================] - 0s 8ms/step - loss: 0.0320 - accuracy: 0.9880 - val_loss: 0.0620 - val_accuracy: 0.9820
Epoch 3/10
42/42 [==============================] - 0s 10ms/step - loss: 0.0074 - accuracy: 0.9970 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 4/10
42/42 [==============================] - 0s 9ms/step - loss: 9.1023e-04 - accuracy: 1.0000 - val_loss: 8.4536e-04 - val_accuracy: 1.0000
Epoch 5/10
42/42 [==============================] - 0s 9ms/step - loss: 1.8245e-04 - accuracy: 1.0000 - val_loss: 3.4866e-04 - val_accuracy: 1.0000
Epoch 6/10
42/42 [==============================] - 0s 9ms/step - loss: 8.2394e-05 - accuracy: 1.0000 - val_loss: 2.0865e-04 - val_accuracy: 1.0000
Epoch 7/10
42/42 [==============================] - 0s 9ms/step - loss: 4.9912e-05 - accuracy: 1.0000 - val_loss: 1.6282e-04 - val_acc

In [ ]:
# Define the augmentations to be applied to the videos
aug = vidaugs.Compose([
    vidaugs.AddNoise(),
    vidaugs.Blur(sigma=5.0),
    vidaugs.OverlayDots(),
])

# Define the paths to the video files
videos_path = "./"
shoot_gun_path = os.path.join(videos_path, "shoot_gun")
brush_hair_path = os.path.join(videos_path, "brush_hair")

# Define the size of the video frames
img_size = 224

# Define the data lists
data_1 = []
labels_1 = []
data_2 = []
labels_2 = []

# Read the first 5 videos from the shoot_gun folder
for i, filename in enumerate(os.listdir(shoot_gun_path)[:5]):
    if filename.endswith(".avi"):
        filepath = os.path.join(shoot_gun_path, filename)
        cap = cv2.VideoCapture(filepath)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (img_size, img_size))
            writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (img_size, img_size), True)
            writer.write(frame)
            writer.release()
            aug_video = cv2.VideoCapture("output.avi")
            aug_frames = []
            while aug_video.isOpened():
                ret, frame = aug_video.read()
                if not ret:
                    break
                aug_frames.append(frame)
            aug_video.release()
            os.remove("output.avi")
            aug_data = np.array(aug_frames)
            data_1.append(aug_data)
        cap.release()

# Read the first 5 videos from the brush_hair folder
for i, filename in enumerate(os.listdir(brush_hair_path)[:5]):
    if filename.endswith(".avi"):
        filepath = os.path.join(brush_hair_path, filename)
        cap = cv2.VideoCapture(filepath)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (img_size, img_size))
            writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (img_size, img_size), True)
            writer.write(frame)
            writer.release()
            aug_video = cv2.VideoCapture("output.avi")
            aug_frames = []
            while aug_video.isOpened():
                ret, frame = aug_video.read()
                if not ret:
                    break
                aug_frames.append(frame)
            aug_video.release()
            os.remove("output.avi")
            aug_data = np.array(aug_frames)
            data_2.append(aug_data)
        cap.release()

print(len(data_1))
print(len(data_2))

# Convert the data lists to numpy arrays
data_1 = np.concatenate(data_1, axis=0)
data_2 = np.concatenate(data_2, axis=0)


# Create the labels for the data
labels_1 = np.zeros((data_1.shape[0],))
labels_2 = np.ones((data_2.shape[0],))

# Merge the data and labels
X = np.concatenate((data_1, data_2))
y = np.concatenate((labels_1, labels_2))

# Shuffle the data and labels
p = np.random.permutation(X.shape[0])
X = X[p]
y = y[p]

# Split the data and labels into training and testing sets
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

448
1213


In [ ]:
# Define the CNN model architecture
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
42/42 [==============================] - 6s 80ms/step - loss: 35.3737 - accuracy: 0.8825 - val_loss: 0.0240 - val_accuracy: 0.9910
Epoch 2/10
42/42 [==============================] - 2s 50ms/step - loss: 0.0094 - accuracy: 0.9955 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 3/10
42/42 [==============================] - 2s 50ms/step - loss: 2.7392e-04 - accuracy: 1.0000 - val_loss: 6.4002e-05 - val_accuracy: 1.0000
Epoch 4/10
42/42 [==============================] - 2s 46ms/step - loss: 1.8313e-04 - accuracy: 1.0000 - val_loss: 1.5768e-05 - val_accuracy: 1.0000
Epoch 5/10
42/42 [==============================] - 2s 49ms/step - loss: 2.4131e-04 - accuracy: 1.0000 - val_loss: 1.8792e-06 - val_accuracy: 1.0000
Epoch 6/10
42/42 [==============================] - 2s 46ms/step - loss: 6.8788e-06 - accuracy: 1.0000 - val_loss: 1.0735e-06 - val_accuracy: 1.0000
Epoch 7/10
42/42 [==============================] - 2s 52ms/step - loss: 5.4180e-05 - accuracy: 1.0000 - val_loss: 5.3359

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

11/11 [==============================] - 0s 20ms/step - loss: 3.0349e-07 - accuracy: 1.0000
Test accuracy: 1.0


### Text

In [ ]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 2s 0us/step


In [ ]:
maxlen = 200
train_data = pad_sequences(train_data, maxlen=maxlen)
test_data = pad_sequences(test_data, maxlen=maxlen)

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [ ]:
embedding_dim = 16

model = Sequential([
    Embedding(10000, embedding_dim, input_length=maxlen),
    GlobalMaxPooling1D(),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=15,
    batch_size=512,
    validation_data=(x_val, y_val)
)

Epoch 1/15
30/30 [==============================] - 8s 229ms/step - loss: 0.6920 - accuracy: 0.5313 - val_loss: 0.6885 - val_accuracy: 0.6819
Epoch 2/15
30/30 [==============================] - 5s 161ms/step - loss: 0.6842 - accuracy: 0.6433 - val_loss: 0.6785 - val_accuracy: 0.7049
Epoch 3/15
30/30 [==============================] - 9s 322ms/step - loss: 0.6691 - accuracy: 0.6845 - val_loss: 0.6598 - val_accuracy: 0.7228
Epoch 4/15
30/30 [==============================] - 6s 182ms/step - loss: 0.6426 - accuracy: 0.7196 - val_loss: 0.6289 - val_accuracy: 0.7351
Epoch 5/15
30/30 [==============================] - 5s 162ms/step - loss: 0.6055 - accuracy: 0.7543 - val_loss: 0.5883 - val_accuracy: 0.7685
Epoch 6/15
30/30 [==============================] - 4s 140ms/step - loss: 0.5597 - accuracy: 0.7886 - val_loss: 0.5451 - val_accuracy: 0.7891
Epoch 7/15
30/30 [==============================] - 4s 146ms/step - loss: 0.5140 - accuracy: 0.8105 - val_loss: 0.5049 - val_accuracy: 0.8029
Epoch 

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 2s 2ms/step - loss: 0.3760 - accuracy: 0.8368
[0.37601831555366516, 0.8367599844932556]


In [ ]:
# Define the augmentation function
def augment_text(text, aug):
    augmented_text = aug.augment(text)
    return augmented_text

# Define the augmentation method
aug = naw.SynonymAug(aug_src='wordnet')

# Augment the text data
augmented_x_train = [aug.augment(str(text)) for text in partial_x_train]
augmented_x_val = [aug.augment(str(text)) for text in x_val]
augmented_test_data = [aug.augment(str(text)) for text in test_data]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Create a tokenizer object
tokenizer = Tokenizer(num_words=10000)
augmented_x_train = tokenizer.texts_to_sequences(augmented_x_train)
augmented_x_val = tokenizer.texts_to_sequences(augmented_x_val)
augmented_test_data = tokenizer.texts_to_sequences(augmented_test_data)

# Pad the augmented data
augmented_x_train = pad_sequences(augmented_x_train, maxlen=maxlen)
augmented_x_val = pad_sequences(augmented_x_val, maxlen=maxlen)
augmented_test_data = pad_sequences(augmented_test_data, maxlen=maxlen)

In [ ]:
# Train the model with augmented data
history = model.fit(
    augmented_x_train,
    partial_y_train,
    epochs=15,
    batch_size=512,
    validation_data=(augmented_x_val, y_val)
)

# Evaluate the model with the augmented test data
results = model.evaluate(augmented_test_data, test_labels)
print(results)

Epoch 1/15
30/30 [==============================] - 1s 8ms/step - loss: 0.7085 - accuracy: 0.5047 - val_loss: 0.6966 - val_accuracy: 0.4947
Epoch 2/15
30/30 [==============================] - 0s 7ms/step - loss: 0.7034 - accuracy: 0.4961 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 3/15
30/30 [==============================] - 0s 8ms/step - loss: 0.6977 - accuracy: 0.4993 - val_loss: 0.6940 - val_accuracy: 0.4947
Epoch 4/15
30/30 [==============================] - 0s 7ms/step - loss: 0.6951 - accuracy: 0.5035 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 5/15
30/30 [==============================] - 0s 8ms/step - loss: 0.6953 - accuracy: 0.5027 - val_loss: 0.6933 - val_accuracy: 0.4947
Epoch 6/15
30/30 [==============================] - 0s 7ms/step - loss: 0.6956 - accuracy: 0.4982 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 7/15
30/30 [==============================] - 0s 7ms/step - loss: 0.6963 - accuracy: 0.4979 - val_loss: 0.6937 - val_accuracy: 0.4947
Epoch 8/15
30/30 [==

With data augmentation(replacing words with synonyms), accuracy reduced from 83% to 50%

### Time Series

In [ ]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
dataset = pd.read_csv(url, header=0, index_col=0)

# Convert the dataset to a numpy array
data = dataset.values

# Split the dataset into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

# Function to create the time series dataset
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Reshape the data into the appropriate format for the neural network
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

# Define the neural network model
model = keras.Sequential([
    keras.layers.LSTM(4, input_shape=(look_back, 1)),
    keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

# Evaluate the model on the testing set
scores = model.evaluate(testX, testY, verbose=0)
print("Test loss:", scores)

Epoch 1/20
2441/2441 - 10s - loss: 52.7120 - 10s/epoch - 4ms/step
Epoch 2/20
2441/2441 - 7s - loss: 15.1355 - 7s/epoch - 3ms/step
Epoch 3/20
2441/2441 - 7s - loss: 9.1413 - 7s/epoch - 3ms/step
Epoch 4/20
2441/2441 - 7s - loss: 7.6507 - 7s/epoch - 3ms/step
Epoch 5/20
2441/2441 - 7s - loss: 7.1739 - 7s/epoch - 3ms/step
Epoch 6/20
2441/2441 - 7s - loss: 6.9644 - 7s/epoch - 3ms/step
Epoch 7/20
2441/2441 - 7s - loss: 6.8852 - 7s/epoch - 3ms/step
Epoch 8/20
2441/2441 - 7s - loss: 6.7769 - 7s/epoch - 3ms/step
Epoch 9/20
2441/2441 - 7s - loss: 6.7320 - 7s/epoch - 3ms/step
Epoch 10/20
2441/2441 - 7s - loss: 6.7097 - 7s/epoch - 3ms/step
Epoch 11/20
2441/2441 - 7s - loss: 6.6441 - 7s/epoch - 3ms/step
Epoch 12/20
2441/2441 - 7s - loss: 6.6974 - 7s/epoch - 3ms/step
Epoch 13/20
2441/2441 - 7s - loss: 6.6336 - 7s/epoch - 3ms/step
Epoch 14/20
2441/2441 - 7s - loss: 6.6410 - 7s/epoch - 3ms/step
Epoch 15/20
2441/2441 - 7s - loss: 6.6012 - 7s/epoch - 3ms/step
Epoch 16/20
2441/2441 - 7s - loss: 6.5920 - 7

In [ ]:
# Define the augmentations
augmenter = (
    TimeWarp() 
    + Drift(max_drift=(0.1, 0.3)) 
    + Reverse()
)

# Apply the augmentations to the training data
trainX_aug = np.zeros_like(trainX)
for i in range(len(trainX)):
    trainX_aug[i, :, 0] = augmenter.augment(trainX[i, :, 0])
    
# Apply the augmentations to the testing data
testX_aug = np.zeros_like(testX)
for i in range(len(testX)):
    testX_aug[i, :, 0] = augmenter.augment(testX[i, :, 0])

In [ ]:
# Train the model on the augmented training data
model.fit(trainX_aug, trainY, epochs=20, batch_size=1, verbose=2)

# Evaluate the model on the augmented testing data
scores_aug = model.evaluate(testX_aug, testY, verbose=0)
print("Test loss (augmented data):", scores_aug)

Epoch 1/20
2441/2441 - 7s - loss: 9.1371 - 7s/epoch - 3ms/step
Epoch 2/20
2441/2441 - 7s - loss: 8.2336 - 7s/epoch - 3ms/step
Epoch 3/20
2441/2441 - 7s - loss: 7.7316 - 7s/epoch - 3ms/step
Epoch 4/20
2441/2441 - 7s - loss: 7.4468 - 7s/epoch - 3ms/step
Epoch 5/20
2441/2441 - 7s - loss: 7.3221 - 7s/epoch - 3ms/step
Epoch 6/20
2441/2441 - 7s - loss: 7.2794 - 7s/epoch - 3ms/step
Epoch 7/20
2441/2441 - 6s - loss: 7.2085 - 6s/epoch - 3ms/step
Epoch 8/20
2441/2441 - 7s - loss: 7.1456 - 7s/epoch - 3ms/step
Epoch 9/20
2441/2441 - 6s - loss: 7.1107 - 6s/epoch - 3ms/step
Epoch 10/20
2441/2441 - 7s - loss: 7.0902 - 7s/epoch - 3ms/step
Epoch 11/20
2441/2441 - 7s - loss: 7.0439 - 7s/epoch - 3ms/step
Epoch 12/20
2441/2441 - 7s - loss: 6.9929 - 7s/epoch - 3ms/step
Epoch 13/20
2441/2441 - 7s - loss: 6.9865 - 7s/epoch - 3ms/step
Epoch 14/20
2441/2441 - 7s - loss: 6.9632 - 7s/epoch - 3ms/step
Epoch 15/20
2441/2441 - 7s - loss: 6.9256 - 7s/epoch - 3ms/step
Epoch 16/20
2441/2441 - 7s - loss: 6.9365 - 7s/ep

### Tabular

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

--2023-04-17 09:09:27--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls’

default of credit c 100%[===================>]   5.28M  3.87MB/s    in 1.4s    

2023-04-17 09:09:29 (3.87 MB/s) - ‘default of credit card clients.xls’ saved [5539328/5539328]



In [ ]:
df = pd.read_excel('default of credit card clients.xls', header=1)

In [ ]:
# Separate the features and target variable
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encode the target variable
encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

In [ ]:
# Before augmentation
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
750/750 [==============================] - 4s 4ms/step - loss: 0.4772 - accuracy: 0.8034 - val_loss: 0.4496 - val_accuracy: 0.8170
Epoch 2/10
750/750 [==============================] - 3s 4ms/step - loss: 0.4432 - accuracy: 0.8181 - val_loss: 0.4415 - val_accuracy: 0.8167
Epoch 3/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4349 - accuracy: 0.8209 - val_loss: 0.4392 - val_accuracy: 0.8173
Epoch 4/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4298 - accuracy: 0.8215 - val_loss: 0.4389 - val_accuracy: 0.8188
Epoch 5/10
750/750 [==============================] - 3s 3ms/step - loss: 0.4274 - accuracy: 0.8211 - val_loss: 0.4414 - val_accuracy: 0.8178
Epoch 6/10
750/750 [==============================] - 3s 4ms/step - loss: 0.4256 - accuracy: 0.8227 - val_loss: 0.4365 - val_accuracy: 0.8190
Epoch 7/10
750/750 [==============================] - 3s 3ms/step - loss: 0.4234 - accuracy: 0.8234 - val_loss: 0.4422 - val_accuracy: 0.8168
Epoch 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

188/188 [==============================] - 0s 2ms/step - loss: 0.4390 - accuracy: 0.8185
Test accuracy: 0.8184999823570251


Data augmentation: let's use noise injection as a data augmentation technique on the credit card dataset.

One way to add noise to the data is by adding random Gaussian noise to each feature with a given mean and sd

In [ ]:
def add_noise(feature, mean, std):
    noise = np.random.normal(mean, std, len(feature))
    return feature + noise

# Add noise to each feature of the dataset
for i in range(1, len(df.columns)):
    df.iloc[:, i] = add_noise(df.iloc[:, i], 0, 0.1)

In [ ]:
# Separate features and labels
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=2)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", score[1])

Epoch 1/20
750/750 - 3s - loss: 0.4810 - accuracy: 0.0000e+00 - 3s/epoch - 4ms/step
Epoch 2/20
750/750 - 2s - loss: 0.4553 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 3/20
750/750 - 2s - loss: 0.4466 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 4/20
750/750 - 2s - loss: 0.4414 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 5/20
750/750 - 2s - loss: 0.4392 - accuracy: 0.0000e+00 - 2s/epoch - 3ms/step
Epoch 6/20
750/750 - 2s - loss: 0.4383 - accuracy: 0.0000e+00 - 2s/epoch - 3ms/step
Epoch 7/20
750/750 - 2s - loss: 0.4363 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 8/20
750/750 - 2s - loss: 0.4362 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 9/20
750/750 - 2s - loss: 0.4346 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 10/20
750/750 - 2s - loss: 0.4311 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 11/20
750/750 - 2s - loss: 0.4309 - accuracy: 0.0000e+00 - 2s/epoch - 2ms/step
Epoch 12/20
750/750 - 2s - loss: 0.4305 - accuracy: 0.0000e+00 - 2s/epoch 

We infer that adding noise to the data can help in making the model more robust to noise in the input data, but it may not always improve the performance of the model. It's always a good idea to try different data augmentation techniques and see which ones work best for your particular problem.

It reduces loss but worsens test accuracy

### Speech

In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
!tar -xf speech_commands_v0.02.tar.gz

--2023-04-17 09:10:50--  https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.128, 142.251.12.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G  22.0MB/s    in 1m 55s  

2023-04-17 09:12:45 (20.2 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]



In [ ]:
data_dir = 'speech_commands_v0.02'
words = ['dog', 'cat']
num_files_per_label = 10

# Create a list of all file paths and corresponding labels
file_paths = []
labels = []
for word in words:
    word_dir = os.path.join(word)
    filenames = os.listdir(word_dir)
    random.shuffle(filenames)  # Shuffle filenames within each label
    filenames = filenames[:num_files_per_label]  # Choose first num_files_per_label filenames
    for filename in filenames:
        file_paths.append(os.path.join(word_dir, filename))
        labels.append(word)

# Shuffle the file paths and labels in unison
zipped = list(zip(file_paths, labels))
random.shuffle(zipped)
file_paths, labels = zip(*zipped)

In [ ]:
# define function to extract features from audio files
def extract_features(file_path):
    # load audio file
    audio_file, sr = librosa.load(file_path)
    # extract Mel-frequency cepstral coefficients (MFCCs) from the audio signal
    mfccs = librosa.feature.mfcc(y=audio_file, sr=sr, n_mfcc=40)
    # pad or truncate the sequence to be of length 174
    pad_width = 174 - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfccs

In [ ]:
cat_folder = './cat'
dog_folder = './dog'

cat_data = []
dog_data = []

# select first 10 files from cat folder
for i, file_name in enumerate(os.listdir(cat_folder)):
    if i == 10:
        break
    file_path = os.path.join(cat_folder, file_name)
    cat_data.append(extract_features(file_path))

# select first 10 files from dog folder
for i, file_name in enumerate(os.listdir(dog_folder)):
    if i == 10:
        break
    file_path = os.path.join(dog_folder, file_name)
    dog_data.append(extract_features(file_path))

cat_data = np.array(cat_data)
dog_data = np.array(dog_data)

# create labels for the data
cat_labels = np.zeros(len(cat_data))
dog_labels = np.ones(len(dog_data))

# concatenate the data and labels
X = np.concatenate((cat_data, dog_data), axis=0)
y = np.concatenate((cat_labels, dog_labels), axis=0)

# reshape data for input to neural network
X = X.reshape(X.shape[0], 40, 174, 1)

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# build the neural network
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(40, 174, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# train the model
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 1.1975 - accuracy: 0.5000 - val_loss: 15.2197 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 36ms/step - loss: 17.6652 - accuracy: 0.5000 - val_loss: 6.8004 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 35ms/step - loss: 7.7032 - accuracy: 0.5000 - val_loss: 1.4649 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 36ms/step - loss: 1.6609 - accuracy: 0.5000 - val_loss: 1.0285 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 36ms/step - loss: 0.9387 - accuracy: 0.5000 - val_loss: 1.2407 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 34ms/step - loss: 1.1258 - accuracy: 0.5000 - val_loss: 0.9535 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 36ms/step - loss: 0.8372 - accuracy: 0.5000 - val_loss: 0.7305 - val_accuracy: 0.5000
Epoch 8/10
1/1 [===========

In [ ]:
# evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

1/1 [==============================] - 0s 25ms/step - loss: 0.8926 - accuracy: 0.5000
Test accuracy: 0.5


In [ ]:
# Afterdata augmentation

def augment(file_path, sample_rate=16000):
    audio_data, _ = librosa.load(file_path, sr=sample_rate)
    aug_audio_data, sr = audaugs.pitch_shift(audio_data, n_steps=10)
    return aug_audio_data

# Load the audio files
cat_files = [os.path.join("./cat", f) for f in os.listdir("./cat") if f.endswith(".wav")]
dog_files = [os.path.join("./dog", f) for f in os.listdir("./dog") if f.endswith(".wav")]

# Augment the audio files
cat_files_aug = [augment(f, sample_rate=16000) for f in cat_files]
dog_files_aug = [augment(f, sample_rate=16000) for f in dog_files]

In [ ]:
def extract_mel_spectrograms(audio_data, sr=16000, n_fft=2048, hop_length=512, n_mels=128):
    mel_spectrograms = []
    for audio in audio_data:
        # Extract Mel spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
        # Convert to decibels
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        mel_spectrograms.append(log_mel_spectrogram)
    return mel_spectrograms

In [ ]:
cat_files_aug_mel = extract_mel_spectrograms(cat_files_aug)
dog_files_aug_mel = extract_mel_spectrograms(dog_files_aug)

In [ ]:
max_time_steps = max([m.shape[1] for m in cat_files_aug_mel + dog_files_aug_mel])

# Pad or truncate each Mel spectrogram to have the same number of time steps
cat_files_aug_mel = np.array([np.pad(m, ((0, 0), (0, max_time_steps - m.shape[1])), mode='constant') for m in cat_files_aug_mel])
dog_files_aug_mel = np.array([np.pad(m, ((0, 0), (0, max_time_steps - m.shape[1])), mode='constant') for m in dog_files_aug_mel])

# Combine cat and dog Mel spectrograms and create labels
X = np.concatenate((cat_files_aug_mel, dog_files_aug_mel), axis=0)
y = np.concatenate((np.zeros(len(cat_files_aug_mel)), np.ones(len(dog_files_aug_mel))), axis=0)

In [ ]:
# Reshape the data to include an extra dimension
X = np.expand_dims(X, axis=-1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = keras.models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the test accuracy
print('Test accuracy:', test_acc)

Epoch 1/10
104/104 [==============================] - 3s 10ms/step - loss: 0.6046 - accuracy: 0.7815 - val_loss: 0.1917 - val_accuracy: 0.9159
Epoch 2/10
104/104 [==============================] - 1s 6ms/step - loss: 0.1545 - accuracy: 0.9348 - val_loss: 0.1416 - val_accuracy: 0.9459
Epoch 3/10
104/104 [==============================] - 1s 6ms/step - loss: 0.1238 - accuracy: 0.9492 - val_loss: 0.1402 - val_accuracy: 0.9459
Epoch 4/10
104/104 [==============================] - 1s 6ms/step - loss: 0.0897 - accuracy: 0.9654 - val_loss: 0.1198 - val_accuracy: 0.9531
Epoch 5/10
104/104 [==============================] - 1s 6ms/step - loss: 0.0817 - accuracy: 0.9675 - val_loss: 0.1782 - val_accuracy: 0.9375
Epoch 6/10
104/104 [==============================] - 1s 8ms/step - loss: 0.0766 - accuracy: 0.9726 - val_loss: 0.2195 - val_accuracy: 0.9183
Epoch 7/10
104/104 [==============================] - 1s 9ms/step - loss: 0.0792 - accuracy: 0.9675 - val_loss: 0.2038 - val_accuracy: 0.9303
Epoch

We can see that accuracy drastically increased from 50% to 96% after augmenting the audio files using pitchshift.

### Document

In [ ]:
!wget http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz
!tar xvzf 20news-18828.tar.gz

Streaming output truncated to the last 5000 lines.
20news-18828/sci.space/61079
20news-18828/sci.space/60809
20news-18828/sci.space/61497
20news-18828/sci.space/60156
20news-18828/sci.space/61475
20news-18828/sci.space/59873
20news-18828/sci.space/61541
20news-18828/sci.space/61489
20news-18828/sci.space/61133
20news-18828/sci.space/62418
20news-18828/sci.space/61289
20news-18828/sci.space/60864
20news-18828/sci.space/61443
20news-18828/sci.space/60189
20news-18828/sci.space/60251
20news-18828/sci.space/61356
20news-18828/sci.space/61255
20news-18828/sci.space/61345
20news-18828/sci.space/61382
20news-18828/sci.space/60847
20news-18828/sci.space/62481
20news-18828/sci.space/60783
20news-18828/sci.space/60854
20news-18828/sci.space/62131
20news-18828/sci.space/60921
20news-18828/sci.space/62317
20news-18828/sci.space/61372
20news-18828/sci.space/61406
20news-18828/sci.space/60247
20news-18828/sci.space/62114
20news-18828/sci.space/60791
20news-18828/sci.space/61362
20news-18828/sci.spac

In [ ]:
# download and load the 20 Newsgroups dataset

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# convert text data to numerical features using a bag-of-words model
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X_train = vectorizer.fit_transform(newsgroups_train.data).toarray()
X_test = vectorizer.transform(newsgroups_test.data).toarray()
y_train = newsgroups_train.target
y_test = newsgroups_test.target

# convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# define a simple neural network model
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(1000, 100)
        self.fc2 = torch.nn.Linear(100, 20)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# create an instance of the neural network model
net = Net()

# define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# train the neural network
for epoch in range(100):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # print statistics
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, loss: {loss.item():.4f}')

# evaluate the neural network on the test data
with torch.no_grad():
    outputs = net(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Test accuracy: {accuracy:.4f}')

Epoch 10, loss: 2.9907
Epoch 20, loss: 2.9880
Epoch 30, loss: 2.9853
Epoch 40, loss: 2.9828
Epoch 50, loss: 2.9802
Epoch 60, loss: 2.9778
Epoch 70, loss: 2.9754
Epoch 80, loss: 2.9731
Epoch 90, loss: 2.9708
Epoch 100, loss: 2.9685
Test accuracy: 0.0944


In [ ]:
# Define the text augmentation function
aug = naw.SynonymAug(aug_src='wordnet')

augmented_data = []
for text in newsgroups_train["data"]:
    augmented_text = aug.augment(text)
    augmented_text = ' '.join(augmented_text)  # Convert list of words to string
    augmented_data.append(augmented_text)

In [ ]:
# Create bag-of-words representations of the augmented text data
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
train_data_bow = vectorizer.fit_transform(augmented_data)
test_data_bow = vectorizer.transform(newsgroups_test.data)

In [ ]:
train_data_bow = train_data_bow.sorted_indices()

In [ ]:
# Define the neural network model

model = Sequential()
model.add(Dense(64, input_shape=(train_data_bow.shape[1],), activation='relu'))
model.add(Dense(20, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
batch_size = 32
epochs = 10
model.fit(train_data_bow, newsgroups_train.target, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/10
354/354 [==============================] - 73s 202ms/step - loss: 1.9406 - accuracy: 0.5733
Epoch 2/10
354/354 [==============================] - 55s 154ms/step - loss: 0.8931 - accuracy: 0.8262
Epoch 3/10
354/354 [==============================] - 41s 116ms/step - loss: 0.5510 - accuracy: 0.8989
Epoch 4/10
354/354 [==============================] - 34s 97ms/step - loss: 0.3752 - accuracy: 0.9342
Epoch 5/10
354/354 [==============================] - 27s 76ms/step - loss: 0.2731 - accuracy: 0.9519
Epoch 6/10
354/354 [==============================] - 21s 58ms/step - loss: 0.2139 - accuracy: 0.9614
Epoch 7/10
354/354 [==============================] - 20s 55ms/step - loss: 0.1778 - accuracy: 0.9653
Epoch 8/10
354/354 [==============================] - 14s 41ms/step - loss: 0.1507 - accuracy: 0.9684
Epoch 9/10
354/354 [==============================] - 15s 43ms/step - loss: 0.1359 - accuracy: 0.9691
Epoch 10/10
354/354 [==============================] - 14s 39ms/step - loss: 0.

In [ ]:
test_loss, test_acc = model.evaluate(test_data_bow, newsgroups_test.target)
print('Test accuracy:', test_acc)

236/236 [==============================] - 1s 4ms/step - loss: 2.0180 - accuracy: 0.6308
Test accuracy: 0.6307753324508667
